**Run all the cells below to make sure everything is working and ready to go. All cells should run without error.**

### Test Matplotlib and Plotting

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
%matplotlib inline

### Test OpenCV

In [3]:
import cv2

## Test Moviepy

In [4]:
# Import everything needed to edit/save/watch video clips
import imageio
imageio.plugins.ffmpeg.download()

from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Find Lanes in Video

In [13]:
new_clip_output = 'test_output_KA.mp4'
clip = VideoFileClip("test.mp4")
print(int(clip.fps*clip.duration))
print(clip.size)
#Filter Parameters
kernel_size=9

#Canny Parameters
lower_treshold =100
higher_treshold =200

#Masked Area Parameters
left_top_corner = (650,420)
right_top_corner = (750, 420)
ignore_mask_color = 255

#Hough parameters

rho = 1
theta = (np.pi/180)
treshold = 3
min_line_length = 10
max_line_gap = 3




def pipeline(image): 
    working_copy = np.copy(image)
    vid_gray = cv2.cvtColor(working_copy,cv2.COLOR_RGB2GRAY) 
    vid_blur_gray = cv2.GaussianBlur(vid_gray,(kernel_size,kernel_size),0)
    #Canny
    vid_edges = cv2.Canny(vid_blur_gray,lower_treshold,higher_treshold)
    
    #Masked Area
    vid_mask = np.zeros_like(vid_edges)
    
    vid_imshape = image.shape
    vid_vertices = np.array([[(0+320,vid_imshape[0]-100),left_top_corner,right_top_corner,(vid_imshape[1]-310,vid_imshape[0]-100)]],dtype=np.int32)
    cv2.fillPoly(vid_mask, vid_vertices,ignore_mask_color)
    vid_masked_edges = cv2.bitwise_and(vid_edges, vid_mask)

    
    #Hough Transform
    #vid_line_image = np.copy(image)*0

    #vid_lines = cv2. HoughLinesP(vid_masked_edges, rho, theta, treshold, np.array([]), min_line_length, max_line_gap)
                 
    #for segment in vid_lines:
        #for x1,y1,x2,y2 in segment:
            # ignoring the horizontal lines
            #if abs(y1-y2) > 10: 
                #cv2.line(vid_line_image,(x1,y1),(x2,y2),(255,0,0),30)
    
    #create a blank (white) copy with the same size
    vid_line_image = np.copy(image)*0
    line_image_r = np.copy(image)*0
    line_image_l= np.copy(image)*0
    #Run Hough

    vid_lines = cv2. HoughLinesP(vid_masked_edges, rho, theta, treshold, np.array([]), min_line_length, max_line_gap)


    # Draw segments

    #for segment in lines:
    #for x1,y1,x2,y2 in segment:
         #if (abs(y1-y2) > 10) and (y1-y2)< -3: 
             #cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    x_r=[]
    y_r=[]
    x_l=[]
    y_l=[]
    z_right=[]
    z_left=[]
    x1_new=[]
    y1_new=[]
    x2_new=[]
    y2_new=[]
    
    
    for segment in vid_lines:
        for x1,y1,x2,y2 in segment:
            # ignoring the horizontal lines and separating left and right
            if (abs(y1-y2) > 10) and (y1-y2)<-2: 
                x_r+=[x1,x2]
                y_r+=[y1,y2]
            elif (abs(y1-y2) > 10) and (y1-y2)> 2:
                x_l+=[x1,x2]
                y_l+=[y1,y2]
            
            
    #Right Lane Border
    if x_r :    
        z_right=np.polyfit(x_r,y_r,1)
        m_r,b_r=z_right
        x1_r= 730
        y1_r= int((x1_r) * m_r + b_r)
        x2_r= 1000
        y2_r= int((x2_r) * m_r + b_r)
        cv2.line(line_image_r,(x1_r,y1_r),(x2_r,y2_r),(255,0,0),10)
    
    #Left Lane Border
    if x_l :
        z_left=np.polyfit(x_l,y_l,1)
        m_l,b_l=z_left
        x1_l=600
        y1_l= int((x1_l) * m_l + b_l)
        x2_l= 300
        y2_l= int((x2_l) * m_l + b_l)
        cv2.line(line_image_l,(x1_l,y1_l),(x2_l,y2_l),(255,0,0),10)

    vid_line_image = cv2.addWeighted(line_image_l, 1, line_image_r,1,0) 
                

    # color_edges

    vid_color_edges = np.dstack((vid_edges,vid_edges,vid_edges))

    #Draw the lines on the line_image
    print(vid_color_edges.shape)
    print(vid_line_image.shape)
    vid_line_edges = cv2.addWeighted(vid_color_edges, 0.1, vid_line_image,1,0)
    
    #Show Masked Area
    vid_color_mask = np.dstack((vid_mask,vid_mask,vid_mask))

    vid_orig_w_polygon = cv2.addWeighted(image, 0.8, vid_color_mask,0.1,0)
    
    
    image = cv2.addWeighted(vid_orig_w_polygon,1, vid_line_edges,1,0)
    
    return image

new_clip =clip.fl_image(pipeline)
new_clip.write_videofile(new_clip_output, audio=False)


250
[1280, 720]
(720, 1280, 3)
(720, 1280, 3)
[MoviePy] >>>> Building video test_output_KA.mp4
[MoviePy] Writing video test_output_KA.mp4


  0%|                                                  | 0/251 [00:00<?, ?it/s]

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


  1%|▎                                         | 2/251 [00:00<00:12, 19.23it/s]

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


...

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)


100%|███████████████████████████████████████▊| 250/251 [00:13<00:00, 18.17it/s]

(720, 1280, 3)
(720, 1280, 3)


100%|████████████████████████████████████████| 251/251 [00:13<00:00, 19.05it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_output_KA.mp4 



In [14]:
HTML("""
<video width="640" height="300" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format(new_clip_output))